In [4]:
import numpy as np
from data_named import data
import matplotlib.pyplot as plt
import statsmodels.api as sm

print(data.columns.values)
print(data.shape)

['Duration (in seconds)' 'UserLanguage' 'participant_number' 'age'
 'gender' 'ethnicity' 'political_orientation' 'political_orientation_text'
 'achieved_education' 'achieved_education_text' 'bothered_pre'
 'uneasy_pre' 'uncomfortable_pre' 'C_time_alone' 'C_academic_performance'
 'T_time_alone' 'T_academic_performance' 'value_activation'
 'bothered_post' 'uneasy_post' 'uncomfortable_post' 'sandwich' 'diet'
 'diet_text' 'reason_for_choice' 'labels_attention' 'feeling_observed'
 'did_value_activation_affect_choice']
(18, 28)


![alternative text](med_analysis.png)

From Rijnhart & Al. 2023 using their equations:

![alternative text](med_analysis_eq.png)

Mediation Analysis Part 1 (eq. 1)

In [5]:
participant_nb = data['participant_number'].to_numpy()
x = np.where(participant_nb > 70, 1, 0)

sandwiches = data['sandwich'].to_numpy()
Y = np.where(sandwiches < 3, 1, 0)

X = sm.add_constant(x)

logit_model = sm.Logit(Y, X)
result = logit_model.fit()

print(result.summary())

Optimization terminated successfully.
         Current function value: 0.623822
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                   18
Model:                          Logit   Df Residuals:                       16
Method:                           MLE   Df Model:                            1
Date:                Wed, 24 May 2023   Pseudo R-squ.:                 0.09191
Time:                        16:35:22   Log-Likelihood:                -11.229
converged:                       True   LL-Null:                       -12.365
Covariance Type:            nonrobust   LLR p-value:                    0.1316
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.0986      0.816     -1.346      0.178      -2.699       0.502
x1             1.5041      1.

Mediation Analysis Part 2 (eq. 2)

In [6]:
M1 = data['bothered_post'].to_numpy() - data['bothered_pre'].to_numpy()
M2 = data['uneasy_post'].to_numpy() - data['uneasy_pre'].to_numpy()
M3 = data['uncomfortable_post'].to_numpy() - data['uncomfortable_pre'].to_numpy()

M = M1 + M2 + M3

participant_nb = data['participant_number'].to_numpy()
x_no_zeros = np.where(participant_nb > 70, 1, -1)
X_no_zeros = sm.add_constant(x_no_zeros)

linear_model = sm.OLS(M, X_no_zeros)
result = linear_model.fit()

print(result.summary())
print(result.pvalues)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                 -0.019
Method:                 Least Squares   F-statistic:                    0.6777
Date:                Wed, 24 May 2023   Prob (F-statistic):              0.422
Time:                        16:35:25   Log-Likelihood:                -93.755
No. Observations:                  18   AIC:                             191.5
Df Residuals:                      16   BIC:                             193.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         12.5375     11.130      1.126      0.2

C:\Users\lucil\anaconda3\envs\ML\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


Mediation Analysis Part 3 (eq. 3)

In [7]:
X = np.column_stack((x, M))

X = sm.add_constant(X)

logit_model = sm.Logit(Y, X)
result = logit_model.fit()

print(result.summary())
print(result.pvalues)

Optimization terminated successfully.
         Current function value: 0.482367
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                   18
Model:                          Logit   Df Residuals:                       15
Method:                           MLE   Df Model:                            2
Date:                Wed, 24 May 2023   Pseudo R-squ.:                  0.2978
Time:                        16:36:16   Log-Likelihood:                -8.6826
converged:                       True   LL-Null:                       -12.365
Covariance Type:            nonrobust   LLR p-value:                   0.02515
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.4400      0.920     -1.565      0.118      -3.243       0.363
x1             1.5791      1.